#### Importing Required Library


In [ ]:
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
from torch import optim
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
from sklearn import metrics
import matplotlib.pyplot as plt
import torchvision

#### Mounting my drive to access the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Providing datapath

In [ ]:
data_path = '/content/drive/MyDrive/Dataset'
batch_size = 32

#### Defining data transforms and data loader

In [ ]:
transform = transforms.Compose([
        transforms.Resize(256, interpolation=3),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

dataset_train = datasets.ImageFolder(os.path.join(data_path, "food"), transform=transform)

train_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

print(f"Data loaded with {len(dataset_train)} train imgs.")


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Data loaded with 479 train imgs.


#### Defining Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            nn.Conv2d(10, 15, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(15),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(15, 10, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
      )

        self.linear_layers = nn.Sequential(
            
          nn.Linear(10 * 28 * 28, 3)
      )

  # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
model = Net()
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(15, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=7840, out_features=3, bias=True)
  )
)


#### Setting up the optimizer and loss function

In [ ]:
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(15, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=7840, out_features=3, bias=True)
  )
)


#### Define epoch and train the model

In [ ]:
epoch = 30

In [ ]:
for i in range(epoch):
    running_loss = 0
    for images, labels in train_loader:
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        # Training pass
        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        #This is where the model learns by backpropagating
        loss.backward()

        #And optimizes its weights here
        optimizer.step()

        running_loss += loss.item()
    print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(train_loader)))

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch 1 - Training loss: 0.025313988452156384
Epoch 2 - Training loss: 0.021342741201321284
Epoch 3 - Training loss: 0.01492061565319697
Epoch 4 - Training loss: 0.031265043374150994
Epoch 5 - Training loss: 0.02179952214161555
Epoch 6 - Training loss: 0.02370212849540015
Epoch 7 - Training loss: 0.009206969384104013
Epoch 8 - Training loss: 0.00887122560137262
Epoch 9 - Training loss: 0.006829369176800052
Epoch 10 - Training loss: 0.011328218132257461
Epoch 11 - Training loss: 0.014299566089175642
Epoch 12 - Training loss: 0.014199318954100211
Epoch 13 - Training loss: 0.041089623079945646
Epoch 14 - Training loss: 0.12007041238248348
Epoch 15 - Training loss: 0.07774036390086016
Epoch 16 - Training loss: 0.033834940272693834
Epoch 17 - Training loss: 0.07370673135543863
Epoch 18 - Training loss: 0.05201400135799001
Epoch 19 - Training loss: 0.02019223803654313
Epoch 20 - Training loss: 0.030335749802179634
Epoch 21 - Training loss: 0.031358201048957805
Epoch 22 - Training loss: 0.012

#### Setting model in eval mode and checking model accuracy

In [ ]:
model.eval()
# getting predictions on test set and measuring the performance
l = []
p = []
for images,labels in train_loader:
    for i in range(len(labels)):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        img = images[i].view(1, 3, 224, 224)
        with torch.no_grad():
            logps = model(img)


        ps = torch.exp(logps)
        probab = list(ps.cpu()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.cpu()[i]
        true_label = true_label.item()
        l.append(true_label)
        p.append(pred_label)

print("Number Of Images Tested =", len(l))
print("\nModel Accuracy = {:.3f}%".format(100*metrics.accuracy_score(l, p)) )

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Number Of Images Tested = 479

Model Accuracy = 98.747%


#### Saving model for future use

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Dataset/food_model.pth')

#### Testing the saved model (no need to run)


In [ ]:
test_model = Net()
test_model.load_state_dict(torch.load('/content/drive/MyDrive/Dataset/food_model.pth'))
if torch.cuda.is_available():
    test_model = test_model.cuda()
test_model.eval()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(10, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(15, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=7840, out_features=3, bias=True)
  )
)

In [ ]:
# getting predictions on test set and measuring the performance
l = []
p = []
for images,labels in train_loader:
    for i in range(len(labels)):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        img = images[i].view(1, 3, 224, 224)
        with torch.no_grad():
            logps = test_model(img)


        ps = torch.exp(logps)
        probab = list(ps.cpu()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.cpu()[i]
        true_label = true_label.item()
        l.append(true_label)
        p.append(pred_label)

print("Number Of Images Tested =", len(l))
print("\nModel Accuracy = {:.3f}%".format(100*metrics.accuracy_score(l, p)) )

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Number Of Images Tested = 479

Model Accuracy = 98.747%
